# VNC Server Setup Guide (Ubuntu/Debian)

## Installation
```bash
sudo apt update
sudo apt install tigervnc-standalone-server xfce4 xfce4-goodies dbus-x11 -y
```

## 🛠 Configuration
### 1. Create VNC User
```bash
sudo adduser vncuser
sudo usermod -aG sudo vncuser
```

### 2. Set VNC Password
```bash
sudo -u vncuser vncpasswd
```

### 3. Create xstartup File
```bash
sudo -u vncuser mkdir -p /home/vncuser/.vnc
echo '#!/bin/sh
unset SESSION_MANAGER
unset DBUS_SESSION_BUS_ADDRESS
exec startxfce4' | sudo -u vncuser tee /home/vncuser/.vnc/xstartup
sudo -u vncuser chmod +x /home/vncuser/.vnc/xstartup
```

## Service Setup
### 1. Create Systemd Service
```bash
sudo nano /etc/systemd/system/vncserver@.service
```
Paste:
```ini
[Unit]
Description=VNC Server
After=network.target

[Service]
Type=simple
User=vncuser
WorkingDirectory=/home/vncuser
PIDFile=/home/vncuser/.vnc/%H:%i.pid
ExecStartPre=/bin/sh -c "/usr/bin/vncserver -kill :%i >/dev/null 2>&1 || :"
ExecStart=/usr/bin/vncserver :%i -geometry 1280x720 -depth 24 -localhost no
ExecStop=/usr/bin/vncserver -kill :%i
Restart=on-failure
RestartSec=5

[Install]
WantedBy=multi-user.target
```

### 2. Enable Service
```bash
sudo systemctl daemon-reload
sudo systemctl enable vncserver@1
sudo systemctl start vncserver@1
```

## Connection
### Via SSH Tunnel
```bash
ssh -L 5901:localhost:5901 your_user@server_ip
```
Then connect VNC client to `localhost:5901`

### Direct Connection (unsafe)
```bash
sudo ufw allow 5901/tcp
```
Connect to `server_ip:5901`

## Troubleshooting
### Common Issues
1. **Black screen**:
   - Reinstall XFCE: `sudo apt install --reinstall xfce4`
   - Check xstartup permissions

2. **Authentication errors**:
   ```bash
   sudo -u vncuser vncpasswd  # Reset password
   sudo chown -R vncuser:vncuser /home/vncuser/.vnc
   ```

3. **Port not listening**:
   ```bash
   sudo systemctl restart vncserver@1
   ss -tulnp | grep 5901
   ```

### View Logs
```bash
journalctl -u vncserver@1 -n 50 --no-pager
```

## Recommended Clients
- [TigerVNC Viewer](https://tigervnc.org/)
- [MobaXterm](https://mobaxterm.mobatek.net/) (Windows)
- Remmina (Linux)

```